In [1]:
from pyspark.context import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf,col,collect_list
from pyspark.sql.types import StringType, ArrayType, DoubleType,IntegerType
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import RankingEvaluator
from HR import HitRate

from pyspark.sql.types import *
from pyspark.sql import functions as F

#For windows user only
import os 
import sys
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

In [2]:
from recommenders.utils.timer import Timer
from recommenders.datasets import movielens
from recommenders.utils.notebook_utils import is_jupyter
from recommenders.datasets.spark_splitters import spark_random_split
from recommenders.evaluation.spark_evaluation import SparkRatingEvaluation, SparkRankingEvaluation
from recommenders.utils.spark_utils import start_or_get_spark

# Create spark session

In [3]:
spark = SparkSession.builder.master("local[*]") \
                    .config('spark.ui.showConsoleProgress', 'false')\
                    .appName('MovieRecomender') \
                    .getOrCreate()
                    
spark.sparkContext.setCheckpointDir('checkpoint/')

22/10/08 17:07:48 WARN Utils: Your hostname, lap15450-ThinkPad-X13-Gen-2i resolves to a loopback address: 127.0.1.1; using 192.168.0.193 instead (on interface wlp0s20f3)
22/10/08 17:07:48 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/10/08 17:07:48 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [4]:
schema =             StructType([
                    StructField('UserID', LongType(), True),
                     StructField('MovieID', LongType(), True),
                     StructField('Rating', IntegerType(), True),
                     StructField('Timestamp', LongType(), True),
                     ])

In [5]:
df = spark.read.option("sep", "::").schema(schema).csv("data/ml-1m/ratings.dat")
df = df.toDF(*["UserID", "MovieID", "Rating", "Timestamp"])
df.createOrReplaceTempView("dataset");
df = df.dropna()
df.cache().count() #Force cache

1000209

# Model config

In [6]:
rankRange = [15, 20]
iterRange = [10, 15]
regParamRange = [0.05, 0.1]

# Hit Rate

In [7]:
def evaluate_hit_rate(als, left_out_df, keep_one_df, full_matrix, n_users):
    hr_evaluator = HitRate(predictionCol='prediction', labelCol='Rating', userCol='UserID', itemCol = "MovieID")
    value = hr_evaluator.eval(als, left_out_df, keep_one_df, full_matrix, n_users)
    return value

# RMSE

In [8]:
def evaluate_rmse(model, test):
    rmse = RegressionEvaluator(metricName="rmse", labelCol="Rating", predictionCol="prediction")       
    predictions=model.transform(test).na.drop()
    return rmse.evaluate(predictions)

# NCDG at K

In [9]:
def getRank(a):
    ret=[]
    for i in a:
        ret.append(float(i.MovieID))
    return ret
def toDouble(a):
    return [float(i) for i in a]

convertUDF = udf(lambda z: getRank(z),ArrayType(DoubleType()))
toDoubleUDF = udf(lambda z: toDouble(z),ArrayType(DoubleType()))

In [10]:
def evaluate_ndcg_at_k(model, full_matrix, train, test, k = 10):
    recommendations = model.transform(full_matrix)
    rank_eval = SparkRankingEvaluation(test, recommendations, k = k, col_user="UserID", col_item="MovieID", 
                                    col_rating="Rating", col_prediction="prediction", 
                                    relevancy_method="top_k")
    return rank_eval.ndcg_at_k()

# Params testing

In [11]:
#Leave one out for calculating hit rate
import pyspark.sql.functions as F
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number

windowSpec  = Window.partitionBy("UserID").orderBy(F.col("Rating").desc())
tmp = df.withColumn("row_number", row_number().over(windowSpec))      
left_out_dataframe = tmp.filter(F.col("row_number") != 1)
keep_one_dataframe = tmp.filter(F.col("row_number") == 1)
left_out_count = left_out_dataframe.persist().count() #Force persist due to size
keep_out_count = keep_one_dataframe.persist().count() #Force persist
print("Left out {}, training to evaluate hit rate on {}.".format(keep_out_count, left_out_count))

Left out 6040, training to evaluate hit rate on 994169.


In [13]:
user_df = df.select("UserID").distinct()
movie_df = df.select("MovieID").distinct()

user_df.cache().count() #Force cache
movie_df.cache().count() #Force cache
full_matrix = user_df.crossJoin(movie_df)
full_matrix.persist().count() #Force persist due to size

#Count
n_users = user_df.count()
n_items = movie_df.count()
for rank in rankRange:
    for iter in iterRange:
        for regParam in regParamRange:
            als = ALS(
                rank=rank,
                maxIter=iter,
                regParam=regParam,
                
                userCol="UserID",
                itemCol="MovieID",
                ratingCol="Rating",
                implicitPrefs=False,
                coldStartStrategy='drop',
                nonnegative=False,
            )
            hit_rate = (evaluate_hit_rate(als, left_out_dataframe, keep_one_dataframe, \
                                     full_matrix, n_users))
            
            # NDCG and RMSE
            
            (train, test) = df.randomSplit([0.8, 0.2])  
            model = als.fit(train)
            ndcg = (evaluate_ndcg_at_k(model, full_matrix, train, test, k = 10))
            rmse = (evaluate_rmse(model, test))
            
            print("Evaluating, rank: {}, iter: {}, regParam: {}".format(rank, iter, regParam))
            print("NDCG: {}, RMSE: {}, Hit: {}".format(ndcg, rmse, hit_rate))

22/10/08 17:09:38 WARN CacheManager: Asked to cache already cached data.
22/10/08 17:09:38 WARN CacheManager: Asked to cache already cached data.
22/10/08 17:09:38 WARN CacheManager: Asked to cache already cached data.


Evaluating, rank: 15, iter: 10, regParam: 0.05
NDCG: 0.029394030721832887, RMSE: 0.8636828510463607, Hit: 0.015066225165562913
Evaluating, rank: 15, iter: 10, regParam: 0.1
NDCG: 0.025936459699574917, RMSE: 0.867699098863675, Hit: 0.014900662251655629
Evaluating, rank: 15, iter: 15, regParam: 0.05
NDCG: 0.026575282300055007, RMSE: 0.8605189280189716, Hit: 0.016721854304635762
Evaluating, rank: 15, iter: 15, regParam: 0.1
NDCG: 0.03728563276210253, RMSE: 0.8644451177771614, Hit: 0.01771523178807947
Evaluating, rank: 20, iter: 10, regParam: 0.05
NDCG: 0.03537428403009448, RMSE: 0.8615501964094113, Hit: 0.015894039735099338
Evaluating, rank: 20, iter: 10, regParam: 0.1
NDCG: 0.032541729791107706, RMSE: 0.8691543228305431, Hit: 0.01639072847682119
Evaluating, rank: 20, iter: 15, regParam: 0.05
NDCG: 0.03341319986290137, RMSE: 0.861504281880866, Hit: 0.017549668874172187
Evaluating, rank: 20, iter: 15, regParam: 0.1
NDCG: 0.035417027163211844, RMSE: 0.8629520446124532, Hit: 0.01804635761589